#import lib

In [ ]:
from keras.applications.vgg16 import VGG16
import tensorflow as tf
import os
import sys

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report


#Load data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
path = '/content/drive/MyDrive/Đồ án KHDL - DS204.N21/Do_an/dataset_pneumonia/'

In [ ]:
PATH = path + 'data_augmentation'
IMG_SIZE = 224
IMG_CHANEL = 3
BATCH_SIZE = 256
COLOR_MODE = 'rgb'

In [ ]:
CLASS_NAME = sorted(os.listdir(PATH + '/train'))

In [ ]:
CLASS_NAME

In [ ]:
train_set = tf.keras.preprocessing.image_dataset_from_directory(
    PATH + '/train',
    labels = 'inferred',
    label_mode = 'binary',
    class_names = CLASS_NAME,
    color_mode = 'rgb',
    batch_size = 256,
    image_size = (IMG_SIZE,IMG_SIZE),
    validation_split=0.2,
    subset='training',
    seed=42,
    interpolation = 'bilinear',
)


In [ ]:
val_set = tf.keras.preprocessing.image_dataset_from_directory(
    PATH + '/train',
    labels='inferred',
    label_mode='binary',
    class_names = CLASS_NAME,
    color_mode='rgb',
    batch_size=256,
    image_size=(IMG_SIZE, IMG_SIZE),
    validation_split=0.2, # Chia tỷ lệ 20% cho validation
    subset='validation',
    seed=42
)

In [ ]:
test_set = tf.keras.preprocessing.image_dataset_from_directory(
    PATH  + '/test',
    labels = 'inferred',
    label_mode = 'binary',
    class_names = sorted(os.listdir(PATH + '/train')),
    color_mode = 'rgb',
    batch_size = 256,
    image_size = (IMG_SIZE,IMG_SIZE),
    interpolation = 'bilinear',
)

# grad-cam

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

# Display
from IPython.display import Image, display
import matplotlib.pyplot as plt
import matplotlib.cm as cm

In [ ]:
from tensorflow import keras
#load model
model = keras.models.load_model('/content/drive/MyDrive/Đồ án KHDL - DS204.N21/Do_an/code_without_segmentation/resnet50_org.h5')

In [ ]:
prediction=model.predict(test_set)
y_pred_total = []
for i in range(0,len(prediction)):
    if prediction[i][0]>prediction[i][1]:  #Printing the prediction of model.
        y_pred_total.append(0)
    else:
        y_pred_total.append(1)


In [ ]:
from glob import glob

In [ ]:
classes = glob('dataset_pneumonia/data_augmentation/train/*')

In [ ]:
y_true = []
for img, label in test_set:
  y_true  += np.array(label).flatten().tolist()

In [ ]:
# Đánh giá mô hình bằng các độ đo: Accuracy, Precision, Recall và F1-score
print("Mô hình DenseNet121 - Bộ dữ liệu")
print("accuracy: ", accuracy_score(y_pred_total, y_true)*100)
print("f1_score: ", f1_score(y_pred_total, y_true, average= 'macro')*100)
print("precision: ", precision_score(y_pred_total, y_true)*100)
print("recall: ", recall_score(y_pred_total, y_true)*100)

In [ ]:
print(classification_report(y_true, y_pred_total, target_names=['NORMAL', 'PNEUMONIA']))

In [ ]:
img_path = '/content/drive/MyDrive/Đồ án KHDL - DS204.N21/Do_an/Đồ án KHDL - DS204.N21/Do_an/dataset_pneumonia/data_augmentation/test/PNEUMONIA/person78_bacteria_380.jpeg'

In [ ]:
size = (224,224)

In [ ]:
def get_img_array(img_path, size):
    # `img` is a PIL image of size 299x299
    img = keras.utils.load_img(img_path, target_size=size)
    # `array` is a float32 Numpy array of shape (299, 299, 3)
    array = keras.utils.img_to_array(img)
    # We add a dimension to transform our array into a "batch"
    # of size (1, 299, 299, 3)
    array = np.expand_dims(array, axis=0)
    return array


def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    # First, we create a model that maps the input image to the activations
    # of the last conv layer as well as the output predictions
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )

    # Then, we compute the gradient of the top predicted class for our input image
    # with respect to the activations of the last conv layer
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    # This is the gradient of the output neuron (top predicted or chosen)
    # with regard to the output feature map of the last conv layer
    grads = tape.gradient(class_channel, last_conv_layer_output)

    # This is a vector where each entry is the mean intensity of the gradient
    # over a specific feature map channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # We multiply each channel in the feature map array
    # by "how important this channel is" with regard to the top predicted class
    # then sum all the channels to obtain the heatmap class activation
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # For visualization purpose, we will also normalize the heatmap between 0 & 1
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

In [ ]:
from tensorflow.keras.applications.inception_v3 import preprocess_input


In [ ]:
img_array = preprocess_input(get_img_array(img_path, size=size))

# Remove last layer's softmax
model.layers[-1].activation = None

# Print what the top predicted class is
preds = model.predict(img_array)
# print("Predicted:", 1 if preds >= 0.5 else 0  )
if preds[0][0]>preds[0][1]:  #Printing the prediction of model.
  print("Predicted:", 0)
else:
  print("Predicted:", 1)
# Generate class activation heatmap
#layer cuối của mô hình cần được đổi tùy theo  mô hình
heatmap = make_gradcam_heatmap(img_array, model, 'concatenate_1')

# Display heatmap
plt.matshow(heatmap)
plt.show()

In [ ]:
preds[0][1]

In [ ]:
def save_and_display_gradcam(img_path, heatmap, cam_path="cam.jpg", alpha=0.4):
    # Load the original image
    img = keras.utils.load_img(img_path)
    img = keras.utils.img_to_array(img)

    # Rescale heatmap to a range 0-255
    heatmap = np.uint8(255 * heatmap)

    # Use jet colormap to colorize heatmap
    jet = cm.get_cmap("jet")

    # Use RGB values of the colormap
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]

    # Create an image with RGB colorized heatmap
    jet_heatmap = keras.utils.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
    jet_heatmap = keras.utils.img_to_array(jet_heatmap)

    # Superimpose the heatmap on original image
    superimposed_img = jet_heatmap * alpha + img
    superimposed_img = keras.utils.array_to_img(superimposed_img)

    # Save the superimposed image
    superimposed_img.save(cam_path)

    # Display Grad CAM
    display(Image(cam_path))

#show 1 ảnh
save_and_display_gradcam(img_path, heatmap)

In [ ]:
path_save_1 = "/content/drive/MyDrive/Đồ án KHDL - DS204.N21/Do_an/output/Grad-cam/inception_v3_aug/dự đoán đúng nhãn 1/"

In [ ]:
path_save_0 = "/content/drive/MyDrive/Đồ án KHDL - DS204.N21/Do_an/output/Grad-cam/inception_v3_aug/dự đoán sai nhãn 0 thành 1/"

In [ ]:
path_save_0 +str(0)+".jpeg"

In [ ]:
#đường dẫn test-set
file_paths = test_set.file_paths


In [ ]:
file_paths

In [ ]:
#Lưu và show tập test
for idx,i in enumerate(y_true):
  if y_pred_total[idx] == i and i == 1:
    save_and_display_gradcam(file_paths[idx],heatmap,cam_path = path_save_1 +str(idx)+".jpeg")
  elif y_pred_total[idx] != i and y_pred_total[idx] ==1:
    save_and_display_gradcam(file_paths[idx],heatmap,cam_path = path_save_0 + str(idx)+".jpeg")
  i+=1